# parses connectivity matrix to js

In [1]:
import pandas as pd
from tqdm import tqdm

connectome = pd.read_csv("paperSupplemental/all-all_connectivity_matrix.csv",index_col=0)
connectome

,29,8419602,9469519,15458316,4393049,4679139,15592696,17327188,15653288,10158972,...,5120097,15690955,15629888,15998913,15730680,15712024,15690392,15690711,5120990,11717035
29,0.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8419602,6.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9469519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15458316,0.0,0.0,0.0,0.0,8.0,15.0,0.0,2.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4393049,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15712024,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15690392,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
15690711,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5120990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
n=0
code = """

print("loading Drosophila melanogaster brain")

var BRAIN=[];
const neuronNodes = 'paperSupplemental/EMS175448-supplement-Supplementary_Data_S2.csv';
const neuronEdges = 'paperSupplemental/all-all_connectivity_matrix.csv'; // Provide the path to your edges CSV file
/* Note: The way these work is sort of confusing
 * After every update, the value in nextState is copied into thisState,
 * and thisState and nextState are swapped (so after the first update, thisState = 1, and nextState = 0) */
BRAIN.thisState = 0;
BRAIN.nextState = 1;


/* Maximum accumulated value that must be exceeded before the Neurite will fire */
BRAIN.fireThreshold = 30;

/* Accumulators are used to decide the value to send to the Left and Right motors of the GoPiGo robot */
/* Since this is the javascript version, you can use these to control whatever you want! */
BRAIN.accumleft = 0;
BRAIN.accumright = 0;

/* Used to remove from Axon firing since muscles cannot fire. */
BRAIN.muscles = [];

BRAIN.muscleList = [];

BRAIN.mLeft = [];
BRAIN.mRight = [];
/* Used to accumulate muscle weighted values in body muscles 07-23 = worm locomotion */
BRAIN.musDleft = [];
BRAIN.musVleft = [];
BRAIN.musDright = [];
BRAIN.musVright = [];
/* Use these to stimulate nose and food sensing neurons */
// we want each simualtion to be slightly different
BRAIN.randExcite = function() {
    for (var i = 0; i < 40; i++) {
        BRAIN.dendriteAccumulate(Object.keys(BRAIN.connectome)[Math.floor(Math.random() * Object.keys(BRAIN.connectome).length)]);
    }
}

BRAIN.setup = function() {
    /* This is the full D melanogaster Connectome as expresed in the form of the connectome
    *  neurite and the postSynaptic neurites. */
    BRAIN.connectome = {};
    BRAIN.postSynaptic = {};

"""

for node in connectome.index:
    n+=1
    code += f"    BRAIN.postSynaptic['{node}'] = [0,0] ;\n"


code += "    print('done with nodes, processing edges')\n"
for FROM,row in tqdm(connectome.iterrows(),total=connectome.shape[0]):
    n+=1

    code += f'    BRAIN.connectome["{FROM}"] = function() {{\n'
    connections = {a:b for a,b in dict(row).items() if b>0}

    for TO, weight in connections.items():
        code += f'        BRAIN.postSynaptic["{TO}"][BRAIN.nextState] += {weight} ; \n'
    r  = f"BRAIN.postSynaptic['ADAL'][BRAIN.nextState] += 2"
    code += f'    }};\n'
code += "    print('done with edges')\n"

code +="}\n\n"
print(n)
with open("connectome.setup.js","w") as f:
    f.write(code)


100%|██████████| 2952/2952 [00:31<00:00, 94.16it/s] 

5904


## input neurons

In [77]:
sensory = pd.read_csv("paperSupplemental\EMS175448-supplement-Supplementary_Data_S2.csv")
sensory = sensory[sensory.celltype =="sensory"]
code =  "BRAIN.neuronsToActicate=[];\n"
for (annotation,cluster), neurons in sensory.groupby(["additional_annotations","level_7_cluster"]):
    code += ( f"BRAIN.neuronsToActicate['R_{annotation}_{cluster}'] = false;\n")
    code += ( f"BRAIN.neuronsToActicate['L_{annotation}_{cluster}'] = false;\n")
code +="\n"
code += "BRAIN.update = function() {\n"

groups = []
for (annotation,cluster), neurons in sensory.groupby(["additional_annotations","level_7_cluster"]):

    code += f"    if (BRAIN.neuronsToActicate['L_{annotation}_{cluster}']){{\n"
    groups.append(f"L_{annotation}_{cluster}")
    for neuron in neurons.left_id:
        if neuron =="no pair":continue
        code += f"        BRAIN.dendriteAccumulate('{neuron}');\n"
    code += f"        BRAIN.runconnectome();\n    }}\n"
    code += f"    if (BRAIN.neuronsToActicate['R_{annotation}_{cluster}']){{\n"
    for neuron in neurons.right_id:
        if neuron =="no pair":continue
        code += f"        BRAIN.dendriteAccumulate('{neuron}');\n"
    code += f"        BRAIN.runconnectome();\n    }}\n"
    groups.append(f"D_{annotation}_{cluster}")

code +="}\n"

with open("connectome.update.js","w") as f:
        f.write(code)



## output neurons

In [89]:
data = pd.read_csv("paperSupplemental\EMS175448-supplement-Supplementary_Data_S2.csv")
output = ["DN-SEZ","DN-VNC","RGN"]
data  = data[data.celltype.isin(output)]
code = "BRAIN.outputgroups=[];\n"
for (annotation,cluster), neurons in sensory.groupby(["additional_annotations","level_7_cluster"]):
    #code += ""
    L = (repr(list(neurons.left_id)))
    R = (repr(list(neurons.right_id)))

    code += f"BRAIN.outputgroups['L_{annotation}_{cluster}'] = {L};\n"
    code += f"BRAIN.outputgroups['R_{annotation}_{cluster}'] = {R};\n"
    
print(code)

BRAIN.outputgroups=[];
BRAIN.outputgroups['L_gustatory-external_11'] = ['3267019', '15730680', '15714173', '15675901', '15678833'];
BRAIN.outputgroups['R_gustatory-external_11'] = ['15612241', '15618756', '15671481', '15615877', '3562954'];
BRAIN.outputgroups['L_gustatory-external_13'] = ['19180425', '15680710', '15678441', '15674083'];
BRAIN.outputgroups['R_gustatory-external_13'] = ['15609427', '15629355', '15611642', '15611791'];
BRAIN.outputgroups['L_gustatory-external_24'] = ['15679484', '15690392', '15715796'];
BRAIN.outputgroups['R_gustatory-external_24'] = ['15610387', '15615305', '15606918'];
BRAIN.outputgroups['L_gustatory-external_3'] = ['15716934', '15714639', '3586984', '4338596', '15716701', '15690093', '15689373', '15674262', '12260696', '15679577', '15938337', '15681740', '15677323', '11745381', '5186461', '18934019'];
BRAIN.outputgroups['R_gustatory-external_3'] = ['15746421', '15595950', '3966683', '15652529', '15556868', '15615042', '15588691', '15556081', '15615589'

In [97]:
data = pd.read_csv("paperSupplemental\EMS175448-supplement-Supplementary_Data_S2.csv")
output = ["DN-SEZ","DN-VNC","RGN"]
output  = data[data.celltype.isin(output)]
output  = repr(list(output.right_id)+list(output.left_id))
Input   = data[data.celltype =="sensory"]
Input  = repr(list(Input.right_id)+list(Input.left_id))


print(f"const output = {output};\nconst input = {Input};\n")

const output = ['14517691', '16393165', '10167116', '8159396', '13580849', '16586265', '8227502', '17659219', '16158918', '2557644', '9750356', '5558724', '12335381', '5478818', '19054481', '17529547', '869360', '6219445', '16057212', '10221622', '5559657', '16463097', '865151', '4116174', '17698681', '10158972', '17355872', '13968203', '3806652', '17355892', '8880781', '858479', '16884962', '7573476', '11905911', '6125806', '3732322', '15599768', '4541081', '10263287', '7589765', '12704360', '5038920', '8741619', '9761454', '15334461', '19508873', '16316981', '3009700', '13167025', '10947427', '3595837', '11261332', '10945795', '13751194', '16115575', '2655798', '5512988', '10945111', '5890323', '15250080', '3598000', '7982324', '5993765', '6114695', '11941255', '4215611', '2670619', '16571210', '9842000', '16434842', '17379420', '5956787', '5038703', '13535387', '3762787', '7571966', '5469039', '7530253', '3727026', '3727004', '5955853', '3762317', '5601924', '7529574', '2260755', '1